In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

examples = [
    {
        "question": "영화 탑건",
        "answer": """
        🛩️👨‍✈️🔥
        """,
    },
    {
        "question": "영화 대부",
        "answer": """
        👨‍👨‍👦🔫🍝
        """,
    },
    {
        "question": "영화 스파이더맨",
        "answer": """
        🚶🏼‍♂️🕷️🦸🏼‍♂️
        """,
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)


memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful AI talking to a human. AI that expresses it with three emojis when a person asks the movie title.",
        ),
        MessagesPlaceholder(variable_name="history"),
        example_prompt,
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [2]:
invoke_chain("영화 Hulk")

content='🟢💥🦵'


In [3]:
invoke_chain("어밴져스")

content='🦸\u200d♂️🦸\u200d♀️🌌'


In [8]:
invoke_chain("제일 처음 물어봤던 영화 제목은?")

content='저는 "Hulk"라는 영화 제목을 물어봤던 것 같아요! 🟢💥🦵'
